In [31]:
import os
import json
from glob import glob
from PIL import Image

import pandas as pd
import numpy as np
import cv2

from utils import read_DSBI_annotation, transform_angelina_label, get_alphabet_map

In [2]:
def label_to_one_hot(label, num_classes=6):
    one_hot = np.zeros(num_classes)
    for i in range(len(label)):
        one_hot[int(label[i]) - 1] = 1
    # join as str
    one_hot = "".join([str(int(i)) for i in one_hot])
    return one_hot

In [3]:
def crop_dsbi_bbox(target_dir="./cropped_images/braille_classification_DSBI"):

    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    path = os.getcwd()
    images_paths = glob(os.path.join(path, "../dataset/DSBI/**/*recto.jpg"))
    labels_paths = glob(os.path.join(path, "../dataset/DSBI/**/*recto.txt"))

    os.makedirs(target_dir, exist_ok=True)

    for i, (image_path) in enumerate(images_paths):
        label_path = image_path.replace(".jpg", ".txt")
        image_name = os.path.basename(image_path)
        image = cv2.imread(image_path)
        width, height = image.shape[1], image.shape[0]
        list_rects = read_DSBI_annotation(label_path, width, height, 0.3, False)

        for rect in list_rects:
            left, top, right, bottom, label = rect
            cropped_image = image[int(top) : int(bottom), int(left) : int(right)]
            im = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
            im = Image.fromarray(im)
            cropped_image_name = image_name.replace(".jpg", f"_{label}.jpg")
            im.save(os.path.join(target_dir, cropped_image_name))


In [29]:
crop_dsbi_bbox()

In [8]:

def crop_angelina_bbox(
    img_path, bbox_path, target_dir="./cropped_images/braille_classification_angelina"
):
    """crop bounding box and save as cropped images with label name"""
    # read image
    img = cv2.imread(img_path)
    # read bbox
    with open(bbox_path, "r") as f:
        bbox = json.load(f)
    # crop and save
    for shape in bbox["shapes"]:
        points = np.array(shape["points"])
        x1 = int(points[:, 0].min())
        y1 = int(points[:, 1].min())
        x2 = int(points[:, 0].max())
        y2 = int(points[:, 1].max())
        # crop
        crop_img = img[y1:y2, x1:x2]
        # label
        label = transform_angelina_label(shape["label"])
        label = label_to_one_hot(label)

        # make dir for output
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)
        # img_path basename + coordinate + label
        save_path = os.path.join(
            target_dir,
            os.path.basename(img_path).replace(".jpg", "")
            + "_"
            + str(x1)
            + "_"
            + str(y1)
            + "_"
            + str(x2)
            + "_"
            + str(y2)
            + "_"
            + label
            + ".jpg",
        )
        try:
            cv2.imwrite(save_path, crop_img)
        except:
            print("error: ", save_path)
            continue


In [25]:
def get_unique_image_names(path):
    """get unique image names from path"""
    images = glob(os.path.join(path, "*.jpg"))
    unique_names = set()
    for image in images:
        #print(image)
        name = os.path.basename(image).split(".jpg")[0]
        #print(name)
        unique_names.add(name)
    return unique_names

In [27]:
path = "../dataset/AngelinaDataset/data"
unique_names = get_unique_image_names(path)
for name in unique_names:
    # get all images with the same name
    images = glob(os.path.join(path, name + "*.jpg"))
    # get all json files with the same name
    jsons = glob(os.path.join(path, name + "*.json"))
    # if there is no json file, continue
    if len(jsons) == 0:
        continue
    # get the first json file
    json_path = jsons[0]
    # get the first image file
    img_path = images[0]
    # crop the bounding box and save as cropped images with label name
    crop_angelina_bbox(img_path, json_path)

СС
СС
СС
СС
СС
СС
СС
СС
СС
СС
error:  ./cropped_images/braille_classification_angelina\16208094668749062583560245145727.labeled_261_-1_279_32_001111.jpg


In [ ]:
# kaggle dataset transform to 000000-111111 version
def transform_kaggle_dataset():


In [42]:
with open("./utils/alphabet_map.json", "r") as f:
        alphabet_map = json.load(f)
classes = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
target_dir = "./cropped_images/braille_classification_kaggle"
if not os.path.exists(target_dir):
        os.makedirs(target_dir)
for class_name in classes:
    label = alphabet_map[class_name.lower()]
    print(class_name)
    # get all images with the same name
    images = glob(os.path.join("../dataset/KaggleDataset", class_name, "*.jpg"))
    for img_path in images:
        if "rot" in img_path:
             continue
        crop_img = cv2.imread(img_path)
        # img_path basename + coordinate + label
        save_path = os.path.join(
            target_dir,
            os.path.basename(img_path).replace(".jpg", "")
            + "_"
            + label
            + ".jpg",
        )
        try:
            cv2.imwrite(save_path,crop_img)
        except:
            print("error: ", save_path)
            continue

A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z
